Sales conribution of each product and of each product category:


In [ ]:

import pandas as pd
import logging


logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO, filename="Sales_by_product_log.log",filemode="w")

payments= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\payment.csv',parse_dates=['payment_date'])
orders= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\orders.csv',parse_dates=['created_at'])
product=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\item.csv',parse_dates=['last_update'])

merged_df=payments.merge(orders,on='payment_id',how='left',indicator=False)
merged_df=merged_df.merge(product,on='item_id',how='left',indicator=False)



merged_df=merged_df.drop(columns=['payment_id','customer_id_x','emp_id','payment_type',
                        'order_id','item_id','customer_id_y','delivery',
                        'created_at','customer_modifications','delivery_notes',
                        'recipe_id',
                        'amount','item_size'])


payment_date_converted=pd.DatetimeIndex(merged_df['payment_date']) 
merged_df['Year']=payment_date_converted.year
merged_df['Year']=merged_df['Year'].astype('str')

merged_df['Month']=payment_date_converted.month 
merged_df['Month']=merged_df['Month'].astype('str')
merged_df['Month'] = merged_df['Month'].str.zfill(2)

merged_df['Year_Month']=merged_df['Year']+'-'+merged_df['Month']

merged_df=merged_df.drop(columns=['payment_date'])

product_category_dictionary={"butter croissant":"food","cinnamon bun":"food", "multigrain toast with butter":"food",
                             "iced coffee":"drinks","hot coffee":"drinks"}
merged_df['product_category']=merged_df['item_name'].map(product_category_dictionary)
merged_df.rename(columns={'item_name':'product_name','item_price':'product_price',
                          'item_quantity':'qty_sold',
                          'Year':'year','Year_Month':'year_month','Month':'month' },inplace=True)


merged_df=merged_df.loc[:,['year','year_month','month',
                           'product_category','product_name',
                           'product_price','qty_sold']]


merged_df['sales']=merged_df['product_price']*merged_df['qty_sold']

monthly_product_sales_df=merged_df.groupby(['product_name','year_month','year','month']).agg(
    monthly_sales=('sales','sum'),
    monthly_qty_sold=('qty_sold','sum'),
).reset_index()

avg_monthly_product_sales_df=monthly_product_sales_df.groupby(['product_name','month']).agg(
    avg_product_sales_by_month_of_the_year=('monthly_sales','mean')
).reset_index()


monthly_product_sales_df=pd.merge(merged_df,monthly_product_sales_df,how='right',on=['product_name',
                                                                                     'year_month','year','month'])
monthly_product_sales_df=monthly_product_sales_df.drop(columns=['sales','qty_sold','monthly_qty_sold','product_price']) 


monthly_product_sales_df=pd.merge(monthly_product_sales_df,avg_monthly_product_sales_df,how='left',on=['product_name','month'] )
monthly_product_sales_df['avg_product_sales_by_month_of_the_year']=monthly_product_sales_df['avg_product_sales_by_month_of_the_year'].apply(lambda x:round(x,2))


monthly_product_sales_df=monthly_product_sales_df.loc[:,['product_name','product_category','year',
                                'year_month','month',
                               'monthly_sales','avg_product_sales_by_month_of_the_year']]


monthly_product_sales_df['monthly_sales']=monthly_product_sales_df['monthly_sales'].apply(lambda x:round(x,2))
monthly_product_sales_df=monthly_product_sales_df.drop_duplicates()

monthly_total_sales_df=monthly_product_sales_df.groupby(['year_month','year','month']).agg(
    total_monthly_sales=('monthly_sales','sum')  
)

sales_by_category_df=monthly_product_sales_df.groupby(['product_category','year_month','year','month']).agg(
    monthly_sales_of_this_product_category=('monthly_sales','sum')
    
)

sales_by_category_df=sales_by_category_df.reset_index()

monthly_product_sales_df=pd.merge(monthly_product_sales_df,sales_by_category_df,how='left', on=['product_category','year_month','year','month'])
total_monthly_sales_df=monthly_product_sales_df.groupby(['year_month','year','month']).agg(
    total_monthly_sales=('monthly_sales','sum')    
)
sales_by_product_df=monthly_product_sales_df.groupby(['product_name','year_month','year','month']).agg(
    product_monthly_sales=('monthly_sales','sum')  
)


sales_by_product_df=sales_by_product_df.reset_index()
monthly_product_sales_df=pd.merge(monthly_product_sales_df,sales_by_product_df,how='left', on=['product_name','year_month','year','month'])
total_monthly_sales_df=total_monthly_sales_df.reset_index()
monthly_product_sales_df=pd.merge(monthly_product_sales_df,total_monthly_sales_df,how='left',on=['year_month','year','month'])

monthly_product_sales_df['percentage_category_sales_from_total_sales']=monthly_product_sales_df['monthly_sales_of_this_product_category']/monthly_product_sales_df['total_monthly_sales']*100
monthly_product_sales_df['percentage_category_sales_from_total_sales']=monthly_product_sales_df['percentage_category_sales_from_total_sales'].apply(lambda x: round(x,2))

annual_sales_by_category_df=merged_df.groupby(['product_category','year']).agg(
    annual_sales_by_category=('sales','sum')
)
annual_sales_by_category_df=annual_sales_by_category_df.reset_index()
annual_sales_total_df=pd.DataFrame(annual_sales_by_category_df.groupby(['year'])['annual_sales_by_category'].sum())
annual_sales_total_df=annual_sales_total_df.reset_index()
annual_sales_total_df=annual_sales_total_df.rename(columns={'annual_sales_by_category':'annual_sales_total'})

annual_sales_by_category_df=pd.merge(annual_sales_by_category_df,annual_sales_total_df,how='left',on='year')
annual_sales_by_category_df['category_sales_perc_from_total']=annual_sales_by_category_df['annual_sales_by_category']/annual_sales_by_category_df['annual_sales_total']
annual_sales_by_category_df['category_sales_perc_from_total']=annual_sales_by_category_df['category_sales_perc_from_total'].apply(lambda x: round(x,2))

annual_sales_by_product_df=merged_df.groupby(['product_name','year']).agg(
    annual_sales_by_product=('sales','sum')
)
annual_sales_by_product_df=annual_sales_by_product_df.reset_index()
annual_sales_total_df=pd.DataFrame(annual_sales_by_product_df.groupby(['year'])['annual_sales_by_product'].sum())
annual_sales_total_df=annual_sales_total_df.reset_index()
annual_sales_total_df=annual_sales_total_df.rename(columns={'annual_sales_by_product':'annual_sales_total'})
annual_sales_total_df.head()
annual_sales_by_product_df=pd.merge(annual_sales_by_product_df,annual_sales_total_df,how='left',on='year')
annual_sales_by_product_df['product_sales_perc_from_total']=annual_sales_by_product_df['annual_sales_by_product']/annual_sales_by_product_df['annual_sales_total']*100
annual_sales_by_product_df['product_sales_perc_from_total']=annual_sales_by_product_df['product_sales_perc_from_total'].apply(lambda x: round(x,2))

#downloading the dataframes to csv files
monthly_product_sales_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Performance_of_each_inventory_item\Monthly_sales_by_product.csv',index=False)
monthly_product_sales_df.to_csv(r'C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Sales by product\Monthly_sales_by_product.csv',index=False)
annual_sales_by_product_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Performance_of_each_inventory_item\Annual_sales_by_product.csv',index=False)
annual_sales_by_product_df.to_csv(r'C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Sales by product\Annual_sales_by_product.csv',index=False)
annual_sales_by_category_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Performance_of_each_inventory_item\Annual_sales_by_product_category.csv',index=False)
annual_sales_by_category_df.to_csv(r'C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Sales by product\Annual_sales_by_product_category.csv',index=False)


Sales generated by different geographic area(LATAM vs USA); country(USA vs Argentina vs Peru);individual headquarters(coffee shop in New York vs coffee shop in Miami etc):

In [ ]:
import pandas as pd
import os

payments= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\payment.csv')
orders= pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\orders.csv')
cafes=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\cafe.csv')
address=pd.read_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\cafe V2\Tables_exported_data\address.csv')

merged_df=payments.merge(orders,on='payment_id',how='left',indicator=True)

payment_date_converted=pd.DatetimeIndex(merged_df['payment_date']) #convert from object to datetime payment_date_converte

merged_df['Year']=payment_date_converted.year
merged_df['Month']=payment_date_converted.month  

merged_df=merged_df.sort_values(by='payment_date',ascending=True)
merged_df=merged_df.drop_duplicates(subset='payment_id',keep='first')#removing duplicates with same payment_id
columns_todrop=['customer_id_x','emp_id','created_at','payment_type', #removing columns that are not needed
                'order_id','item_id','customer_id_y','delivery',
                'delivery_address_id','item_quantity',
                'customer_modifications','delivery_notes','_merge']
merged_df=merged_df.drop(columns=columns_todrop,axis=1)

columns=['cafe_id','payment_date','Year','Month','payment_id','amount']
merged_df=merged_df[columns]
merged_df['payment_date']=pd.to_datetime(merged_df['payment_date'])
merged_df['Year']=merged_df['Year'].astype('str')
merged_df['Month']=merged_df['Month'].astype('str')
#using .loc to change all values in column 'Month' to 2 digits
merged_df['Month'] = merged_df['Month'].str.zfill(2)
merged_df['Year_Month']=merged_df['Year']+'-'+merged_df['Month']

monthly_sales_df=merged_df.groupby(['cafe_id','Year_Month','Year','Month']).agg(
    monthly_revenue_total=('amount','sum')
).reset_index()

monthly_sales_df=monthly_sales_df.reset_index(drop=True)

cafe_address_df=pd.merge(cafes,address,on='address_id',how='left')
sales_df=pd.merge(monthly_sales_df,cafe_address_df,on='cafe_id',how='left')

sales_df=sales_df.drop(columns=['manager_emp_id','phone_number','address_id','last_update_x','address_first_line',
                         'address_id','address_second_line','postal_code','last_update_y','city'],axis=1)


#changing cafe_id 1,2,3,4,5 TO "New York","Miami","Lima","Buenos Aires","Seattle"
cafe_id_dictionary ={1: "New York", 2: "Miami", 3: "Lima",4: "Buenos Aires",5: "Seattle"}
sales_df['cafe_id'] = sales_df['cafe_id'].map(cafe_id_dictionary)
#adding column "Region" 
region_dictionary={"USA":"North America","Peru":"LATAM","Argentina":"LATAM"}
sales_df["region"]=sales_df['country'].map(region_dictionary)
sales_df=sales_df.loc[:,['cafe_id','country','region','Year_Month','Year','Month','monthly_revenue_total']]


monthly_sales_by_venue_df=sales_df

monthly_sales_by_venue_avg_df=monthly_sales_by_venue_df.groupby(['cafe_id','Month']).agg(
   avg_revenue_month_of_the_year_by_location=('monthly_revenue_total','mean')
).reset_index()
monthly_sales_by_venue_avg_df['avg_revenue_month_of_the_year_by_location']=pd.DataFrame(monthly_sales_by_venue_avg_df['avg_revenue_month_of_the_year_by_location'].apply(lambda x:round(x,2)))
monthly_sales_by_venue_df=pd.merge(monthly_sales_by_venue_df,monthly_sales_by_venue_avg_df,how='left',on=['Month','cafe_id'])
monthly_sales_by_venue_df.reset_index()

#calculating  average monthly revenue in each location 
monthly_sales_by_venue_avg_df=monthly_sales_by_venue_df.groupby(['cafe_id']).agg(
   avg_monthly_revenue_by_location=('monthly_revenue_total','mean')
).reset_index()

monthly_sales_by_venue_avg_df['avg_monthly_revenue_by_location']=pd.DataFrame(monthly_sales_by_venue_avg_df['avg_monthly_revenue_by_location'].apply(lambda x:round(x,2)))

monthly_sales_by_venue_df=pd.merge(monthly_sales_by_venue_df,monthly_sales_by_venue_avg_df,how='left',on=['cafe_id'])
monthly_sales_by_venue_df.reset_index()
monthly_sales_by_venue_df=pd.merge(monthly_sales_by_venue_df,monthly_sales_by_venue_avg_df,how='left',on=['cafe_id'])
monthly_sales_by_venue_df=monthly_sales_by_venue_df.reset_index(drop=True)
monthly_sales_by_venue_df=monthly_sales_by_venue_df.drop(columns=['avg_monthly_revenue_by_location_y'])
monthly_sales_by_venue_df=monthly_sales_by_venue_df.rename(columns={'avg_monthly_revenue_by_location_x':'avg_monthly_revenue_by_location'})

#"C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Revenue by geographical location\New Microsoft Excel Worksheet.xlsx"
#"C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Sales by geographical location\Sales_by_region_country_venue_all_same_workbook.ipynb"

monthly_sales_by_venue_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Sales by geographical location\Monthly_sales_by_venue.csv',index=False)
monthly_sales_by_venue_df.to_csv(r"C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Sales by geographical location\Monthly_sales_by_venue.csv",index=False)

Annual_sales_by_country_df=pd.DataFrame(sales_df.groupby(['country','Year'])['monthly_revenue_total'].sum())

Annual_sales_by_country_df=Annual_sales_by_country_df.reset_index()
Annual_sales_by_country_df=Annual_sales_by_country_df.rename(columns={'monthly_revenue_total':'annual_revenue_by_country'})
Total_annual_revenue_df=pd.DataFrame(Annual_sales_by_country_df.groupby(['Year']).agg(
    total_annual_revenue=('annual_revenue_by_country','sum')
)).reset_index()

Annual_sales_by_country_df=pd.merge(Annual_sales_by_country_df,Total_annual_revenue_df,how='left',on='Year')
Annual_sales_by_country_df['percentage_from_total_annual_revenue']=(Annual_sales_by_country_df['annual_revenue_by_country']/Annual_sales_by_country_df['total_annual_revenue']*100).apply(lambda x:round(x,2))
Annual_sales_by_country_df=Annual_sales_by_country_df.sort_values(by='Year')


Annual_sales_by_country_df["region"]=Annual_sales_by_country_df['country'].map(region_dictionary)

Annual_sales_by_country_df=Annual_sales_by_country_df.loc[:,['region','country','Year','annual_revenue_by_country','total_annual_revenue','percentage_from_total_annual_revenue']]


Annual_sales_by_country_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Sales by geographical location\Annual_sales_by_country.csv',index=False)
Annual_sales_by_country_df.to_csv(r'C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Sales by geographical location\Annual_sales_by_country.csv',index=False)

Annual_sales_by_region_df=pd.DataFrame(sales_df.groupby(['region','Year'])['monthly_revenue_total'].sum())
Annual_sales_by_region_df=Annual_sales_by_region_df.reset_index()
Annual_sales_by_region_df=Annual_sales_by_region_df.rename(columns={'monthly_revenue_total':'annual_revenue_by_region'})

annual_total_df=pd.DataFrame(Annual_sales_by_region_df.groupby(['Year'])['annual_revenue_by_region'].sum())
annual_total_df=annual_total_df.reset_index()
annual_total_df=annual_total_df.rename(columns={'annual_revenue_by_region':'total_annual_revenue'})

Annual_sales_by_region_df=pd.merge(Annual_sales_by_region_df,annual_total_df, on='Year',how='left')
Annual_sales_by_region_df['percentage_from_total_annual_revenue']=Annual_sales_by_region_df['annual_revenue_by_region']/Annual_sales_by_region_df['total_annual_revenue']*100
Annual_sales_by_region_df['percentage_from_total_annual_revenue']=Annual_sales_by_region_df['percentage_from_total_annual_revenue'].apply(lambda x: round(x,2))

Annual_sales_by_region_df.to_csv(r'C:\Users\SOFYA\OneDrive\Desktop\SQL learning resources\cafe chain project\Tables for analysis\Sales by geographical location\Sales_by_region_North_America_vs_LATAM.csv',index=False)
Annual_sales_by_region_df.to_csv(r'C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Sales by geographical location\Sales_by_region_North_America_vs_LATAM.csv',index=False)

#Compiling .csv files into single .xlsx file to later use in Tableau

csv_folder_path = "Sales by geographical location"
output_excel_path = r"C:\Users\SOFYA\OneDrive\Documents\My Tableau Repository\Datasources\cafe project\Sales by geographical location\Sales_.xlsx"

with pd.ExcelWriter(output_excel_path, engine='openpyxl') as writer:
    for csv_file in os.listdir(csv_folder_path):
        if csv_file.endswith('.csv'):
            df = pd.read_csv(os.path.join(csv_folder_path, csv_file))
            sheet_name = os.path.splitext(csv_file)[0]
            df.to_excel(writer, sheet_name=sheet_name, index=False)

